<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/Astra_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

# Загрузка данных
data = pd.read_excel('/content/Data_Set4.xlsx')
data.replace('н.д.', pd.NA, inplace=True)
data['Возраст'].fillna(data['Возраст'].median(), inplace=True)

# Очистка данных от пробелов
data['Пол'] = data['Пол'].str.strip()
data['Был ли осуществлен привод'] = data['Был ли осуществлен привод'].str.strip()

# Инициализация LabelEncoder для каждого категориального столбца и его обучение
le_dict = {}
for column in ['Пол', 'Роль в суде', 'Рецидив']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    le_dict[column] = le

# Кодирование целевой переменной
le_target = LabelEncoder()
data['Был ли осуществлен привод'] = le_target.fit_transform(data['Был ли осуществлен привод'])

# Целевая переменная и признаки
X = data.drop('Был ли осуществлен привод', axis=1)
y = data['Был ли осуществлен привод']

# Балансировка классов с помощью SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Настройка гиперпараметров с помощью GridSearchCV для XGBoost
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

model = XGBClassifier(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print(f"Лучшие параметры: {grid_search.best_params_}")

# Оценка модели
predictions = best_model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Точность модели: {accuracy:.2f}")

print("Отчет о классификации:")
print(classification_report(y_test, predictions, zero_division=0))

# Вычисление ROC-AUC для мультиклассовой задачи
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))
roc_auc = roc_auc_score(y_test_binarized, best_model.predict_proba(X_test), multi_class='ovr')
print(f"ROC-AUC: {roc_auc:.2f}")

# Создание DataFrame для хранения предсказаний
predictions_df = pd.DataFrame(columns=['Пол', 'Возраст', 'Роль в суде', 'Рецидив', 'Вероятность посещения'])

# Определение возрастных групп
age_groups = [(16, 20), (20, 25), (25, 30), (30, 35), (35, 40), (40, 45), (45, 50), (50, 55), (55, 60), (60, 65), (65, 70), (70, 80)]

# Генерация предсказаний для различных комбинаций пола, возраста, роли в суде и рецидива
genders = le_dict['Пол'].inverse_transform([0, 1])
roles = le_dict['Роль в суде'].inverse_transform([0, 1, 2])
recidivisms = le_dict['Рецидив'].inverse_transform([0, 1])

for gender in genders:
    for age_group in age_groups:
        for role in roles:
            for recidivism in recidivisms:
                age_mean = np.mean(age_group)
                new_data = pd.DataFrame({
                    'Пол': [le_dict['Пол'].transform([gender])[0]],
                    'Возраст': [age_mean],
                    'Роль в суде': [le_dict['Роль в суде'].transform([role])[0]],
                    'Рецидив': [le_dict['Рецидив'].transform([recidivism])[0]]
                })

                new_prediction = best_model.predict_proba(new_data)[:, 1]  # Вероятность посещения (класс 1)
                new_prediction_percent = round(new_prediction[0] * 100, 2)  # Преобразование в проценты и округление до двух знаков

                predictions_df = pd.concat([predictions_df, pd.DataFrame({
                    'Пол': [gender],'Возраст': [f"{age_group[0]}-{age_group[1]}"],
                    'Роль в суде': [role],
                    'Рецидив': [recidivism],
                    'Вероятность посещения (%)': [new_prediction_percent]
                })], ignore_index=True)

# Сохранение предсказаний в файл CSV
predictions_df.to_csv('Astra.csv', index=False)
print("Файл Astra.csv успешно создан.")

Лучшие параметры: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}
Точность модели: 0.84
Отчет о классификации:
              precision    recall  f1-score   support

           0       0.84      0.74      0.79       494
           1       0.83      1.00      0.91       481
           2       0.84      0.78      0.81       537

    accuracy                           0.84      1512
   macro avg       0.84      0.84      0.83      1512
weighted avg       0.84      0.84      0.83      1512

ROC-AUC: 0.96
Файл Astra.csv успешно создан.
